In [1]:
import numpy as np
import scipy
import cv2 as cv
# from google.colab.patches import cv2_imshow

location = 'C:/Users/user/datasets/cats_0.jpg'
img1 = cv.imread(location)
print(img1.shape)
if img1 is None:
  print('Image load failed!')

flip_down = np.flipud(img1)

cv.imshow('test', flip_down)
cv.imwrite('C:/Users/user/datasets/dog.5.jpg', flip_down)
cv.imshow('test', flip_down)
cv.waitKey(0)
cv.destroyAllWindows()

(240, 174, 3)


In [2]:
import os
location = 'C:/Users/user/datasets/'
image_list = os.listdir(location)

In [3]:
image_list

['.ipynb_checkpoints',
 'cats_0.jpg',
 'cats_1.jpg',
 'cats_2.jpg',
 'cats_3.jpg',
 'cats_4.jpg',
 'cats_5.jpg',
 'cats_6.jpg',
 'cats_7.jpg',
 'cats_8.jpg',
 'cats_9.jpg',
 'cats_and_dogs_small',
 'dog.5.jpg',
 'test',
 'train',
 'validation']

In [4]:
index = np.random.choice(len(image_list), 5)
print(index)

[ 9 14  6  2 11]


In [5]:
from scipy import ndimage
import numpy as np
import scipy
import cv2 as cv

index = np.random.choice(len(image_list), 5, replace=False)

count = 0
for i in index:
  
  location = 'C:/Users/user/datasets/cats_{}.jpg'.format(i)
  print(location)
  
  image = cv.imread(location)
  
  if i % 2 == 0:
    image = np.flipud(image)
  elif i % 3 == 0 :
    image = ndimage.rotate(image, np.random.choice(45, 1)[0])
  elif i % 5 == 0:
    image = ndimage.gaussian_filter(image, sigma=np.random.choice(5, 1)[0])
    
  location = 'C:/Users/user/datasets/cats_{}.jpg'.format((len(index)+count))
  cv.imwrite(location, image)
  count += 1

C:/Users/user/datasets/cats_4.jpg
C:/Users/user/datasets/cats_7.jpg
C:/Users/user/datasets/cats_8.jpg
C:/Users/user/datasets/cats_13.jpg
C:/Users/user/datasets/cats_6.jpg


In [6]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

# 모든 이미지를 1/255로 스케일을 조정합니다
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

base_dir = 'C:/Users/user/datasets/'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')


train_generator = train_datagen.flow_from_directory(
        # 타깃 디렉터리
        train_dir,
        # 모든 이미지를 224 × 224 크기로 바꿉니다
        target_size=(224, 224),
        batch_size=20,
        # binary_crossentropy 손실을 사용하기 때문에 이진 레이블이 필요합니다
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=20,
        class_mode='binary')

Using TensorFlow backend.


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [7]:
for data_batch, labels_batch in train_generator:
    print('Batch size:', data_batch.shape)
    print('Label size:', labels_batch.shape)
    break

Batch size: (0, 224, 224, 3)
Label size: (0,)


In [8]:
#  This code cell shows how to utilize VGG16 model by combining Dense layer at the end of the network 
from keras.layers import Input, Dense, GlobalAveragePooling2D, Flatten
from keras.models import Model

from keras.applications.vgg16 import VGG16
from keras import layers
from keras import models

VGGNet = VGG16()

# VGGNet.summary()

# We will not update VGG pre-trained model, only added Dense layers will be trained from the scratch
for layer in VGGNet.layers:
  layer.trainable = False
  
vgg_maxpool5 = VGGNet.get_layer('block5_pool').output

Feature_Flatten = Flatten()(vgg_maxpool5)
dense = Dense(10, name='dense', activation='relu')(Feature_Flatten)
predictions = Dense(1, activation='sigmoid')(dense)
New_VGGmodel = Model(inputs=VGGNet.input, outputs=predictions)
New_VGGmodel.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

New_VGGmodel.summary()








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
________________________

In [ ]:
history = New_VGGmodel.fit_generator(
      train_generator,
      steps_per_epoch=10,
      epochs=30,
      )

Epoch 1/30


In [ ]:
New_VGGmodel.save('MyVGGNet.h5')